<a href="https://colab.research.google.com/github/aromalsr/Thesis/blob/main/autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
from pathlib import Path
from datetime import datetime
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib import cm
import random
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error as mse
from sklearn.neighbors import KernelDensity

from keras.models import Model
from tensorflow.keras.layers import Conv2D, Activation, Dense, Dropout, Softmax, LeakyReLU, BatchNormalization, AveragePooling2D, Reshape, Flatten, Conv2DTranspose
from tensorflow.keras.optimizers import RMSprop, Adam
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, LearningRateScheduler
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy, SparseCategoricalCrossentropy, MeanSquaredError

import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image


In [ ]:
# Size of our input images and no of channels
SIZE = 128
no_channels = 1

#Dataset Path

TRAIN_DATASET_PATH = Path("/content/enamel_dataset/good_images/GoodImg1")
# VALID_DATASET_PATH = Path("/content/enamel_dataset/ValidImg")
PRED1_DATASET_PATH = Path("/content/enamel_dataset/pred_goodImg/pred_1")                      #Dataset without defect
PRED2_DATASET_PATH = Path("/content/enamel_dataset/blob_defect/blobs")                       #Dataset with defect
PRED3_DATASET_PATH = Path("/content/enamel_dataset/scratch_defect/scratches")                   #Dataset with defect

img_paths = list(TRAIN_DATASET_PATH.glob("**/*.bmp"))
train_img_paths = img_paths[0:800]
valid_img_paths = img_paths[801:1000]
pred1_img_paths = list(PRED1_DATASET_PATH.glob("**/*.bmp"))
pred2_img_paths = list(PRED2_DATASET_PATH.glob("**/*.bmp"))
pred3_img_paths = list(PRED3_DATASET_PATH.glob("**/*.bmp"))
#pred4_img_paths = list(PRED2_DATASET_PATH.glob("**/*.bmp"))

In [ ]:
def get_image(img_path):
    img_array = []

    for img in img_path:

        image = cv2.imread(str(img),0)
        # image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        # image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        #image = cv2.resize(image,(128,128),interpolation=cv2.INTER_AREA)
        image = cv2.resize(image, (128, 128))
        image = image/255

        img_array.append(image)

    print('image shape: ', image.shape)

    image_np = np.asarray(img_array)
    image_np = tf.expand_dims(image_np, axis=-1)
    print('shape of input nparray:',image_np.shape)

    return image_np

In [ ]:
print("training data:")
x_train = get_image(train_img_paths)
print("test data:")
x_test = get_image(valid_img_paths)

x_train[1].shape

In [ ]:
x_Train = x_train.numpy()
score = ssim(x_Train[100], x_Train[100], full=True, multichannel=True)
print(score[0])

In [ ]:
 random_num = random.randint(0,len(x_train) - 1)
#  plt.imshow(x_train[random_num], cmap="gray")
 x_train_test = tf.squeeze(x_train, axis= -1)
 print('shape of input nparray:',x_train_test.shape)

 plt.imshow(x_train_test[random_num], cmap="gray")
 print("image_number = ",random_num)

In [ ]:


# Create the Encoder and Decoder
#Encoder
#pass the gray scale input image of size(128,128,1)
inputs = tf.keras.Input(shape=(SIZE,SIZE, 1), name='input_layer')
# Conv Block 1 -> BatchNorm->leaky Relu
encoded = Conv2D(256, kernel_size=3, strides= 1, padding='same', name='conv_1')(inputs)
encoded = BatchNormalization(name='batchnorm_1')(encoded)
encoded = LeakyReLU(name='relu_1')(encoded)
# Conv Block 2 -> BatchNorm->leaky Relu
encoded = Conv2D(128, kernel_size=3, strides= 1, padding='same', name='conv_2')(encoded)
encoded = BatchNormalization(name='batchnorm_2')(encoded)
encoded = LeakyReLU(name='relu_2')(encoded)
# Conv Block 3 -> BatchNorm->leaky Relu
encoded = Conv2D(64, kernel_size=3, strides= 2, padding='same', name='conv_3')(encoded)
encoded = BatchNormalization(name='batchnorm_3')(encoded)
encoded = LeakyReLU(name='relu_3')(encoded)
# Conv Block 4 -> BatchNorm->leaky Relu
encoded = Conv2D(32, kernel_size=3, strides=2, padding='same', name='conv_4')(encoded)
encoded = BatchNormalization(name='batchnorm_4')(encoded)
encoded = LeakyReLU(name='relu_4')(encoded)
# Conv Block 5 -> BatchNorm->leaky Relu
encoded = Conv2D(16, kernel_size=3, strides=2, padding='same', name='conv_5')(encoded)
encoded = BatchNormalization(name='batchnorm_5')(encoded)
encoded = LeakyReLU(name='relu_5')(encoded)

#Decoder
# DeConv Block 1-> BatchNorm->leaky Relu
decoded = Conv2DTranspose(16, 3, strides= 1, padding='same',name='conv_transpose_1')(encoded)
decoded = BatchNormalization(name='batchnorm_6')(decoded)
decoded = LeakyReLU(name='relu_6')(decoded)
# DeConv Block 2-> BatchNorm->leaky Relu
decoded = Conv2DTranspose(32, 3, strides= 2, padding='same', name='conv_transpose_2')(decoded)
decoded = BatchNormalization(name='batchnorm_7')(decoded)
decoded = LeakyReLU(name='relu_7')(decoded)
# DeConv Block 3-> BatchNorm->leaky Relu
decoded = Conv2DTranspose(64, 3, 2, padding='same', name='conv_transpose_3')(decoded)
decoded = BatchNormalization(name='batchnorm_8')(decoded)
decoded = LeakyReLU(name='relu_8')(decoded)
# DeConv Block 4-> BatchNorm->leaky Relu
decoded = Conv2DTranspose(128, 3, 2, padding='same', name='conv_transpose_4')(decoded)
decoded = BatchNormalization(name='batchnorm_9')(decoded)
decoded = LeakyReLU(name='relu_9')(decoded)

 # output
outputs = Conv2DTranspose(no_channels, 3, 1,padding='same', activation='sigmoid', name='conv_transpose_10')(decoded)

In [ ]:
def SSIMLoss(y_true, y_pred):
   return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred,1.0,filter_size=7))

autoencoder = Model(inputs, outputs)
autoencoder.compile(optimizer=Adam(lr = 0.001),loss=SSIMLoss, metrics=['mse'])

autoencoder.output_shape

In [ ]:
history=autoencoder.fit(x_train, x_train,
                epochs=35,
                batch_size=32,
                shuffle=True,
                validation_data=(x_test, x_test) )

# history_2 = autoencoder.fit(
#     train_generator,
#     steps_per_epoch=500 // batch_size,
#     epochs=50,
#     validation_data=validation_generator,
#     validation_steps=75 // batch_size,
#     shuffle=True
#     )

In [ ]:
# plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

## Dataset Without Anomaly

In [ ]:
x_pred1 = get_image(pred1_img_paths)
x_Pred1 = x_pred1.numpy()                 # not sure | done to solve the error occured with ssim function

decoded_imgs_without_anomaly = autoencoder.predict(x_pred1)
# decoded_imgs_without_Anomaly = decoded_imgs_without_anomaly.numpy
show_result(x_pred1,decoded_imgs_without_anomaly)

## Dataset With Anomaly

In [ ]:
x_pred2 = get_image(pred2_img_paths)
x_Pred2 = x_pred2.numpy()


decoded_imgs_with_blob_anomaly = autoencoder.predict(x_pred2)
show_result(x_pred2,decoded_imgs_with_blob_anomaly)

In [ ]:
x_pred3 = get_image(pred3_img_paths)
x_Pred3 = x_pred3.numpy()


decoded_imgs_with_scratch_anomaly = autoencoder.predict(x_pred3)
show_result(x_pred3,decoded_imgs_with_scratch_anomaly)

## Calculating SSIM Loss for each images

In [ ]:
# ssim_with_anomaly_array = np.asarray([ssim(x_Pred2[i], decoded_imgs_with_anomoly[i], multichannel = True) for i in range(0, x_Pred2.shape[0])]).reshape(-1,1)
ssim_with_blob_anomaly_array = np.asarray([ssim(x_Pred2[i], decoded_imgs_with_blob_anomaly[i], multichannel = True) for i in range(0, x_Pred2.shape[0])]).reshape(-1,1)
ssim_with_scratch_anomaly_array = np.asarray([ssim(x_Pred3[i], decoded_imgs_with_scratch_anomaly[i], multichannel = True) for i in range(0, x_Pred3.shape[0])]).reshape(-1,1)
ssim_without_anomaly_array = np.asarray([ssim(x_Pred1[i], decoded_imgs_without_anomaly[i], multichannel = True) for i in range(0, x_Pred1.shape[0])]).reshape(-1,1)

### SSIM for With & Without Anomaly Images

In [ ]:
plt.subplots(figsize= (10,7))
plt.title("SSIM")
# plt.scatter(x=np.arange(0,len(ssim_with_anomaly_array)),y=ssim_with_anomaly_array, label='ssim with anomaly')
plt.scatter(x=np.arange(0,len(ssim_with_blob_anomaly_array)),y=ssim_with_blob_anomaly_array, label='ssim with blob anomaly')
plt.scatter(x=np.arange(0,len(ssim_with_scratch_anomaly_array)),y=ssim_with_scratch_anomaly_array, label='ssim with scratch anomaly')
plt.scatter(x=np.arange(0,len(ssim_without_anomaly_array)),y=ssim_without_anomaly_array, label='ssim without anomaly')
plt.legend()
plt.tight_layout()
plt.show()

## Squeezed Result

In [ ]:
def get_error_ssim(imageA, imageB):

    fragmented_image_error = []
    window = 20
    temp_error = 1

    for x in range(0, imageA.shape[0]-window+1, int(window/2)):
        for y in range(0, imageA.shape[1]-window+1, int(window/2)):
            error = ssim(imageA[x : x + window, y : y + window],
                        imageB[x : x + window, y : y + window])
            fragmented_image_error.append(error)
            if error < temp_error:
                pos = (x,y)
                temp_error = error

    return temp_error, pos, window

In [ ]:
X_pred1=np.squeeze(x_pred1)
Decoded_imgs_without_anomaly = np.squeeze(decoded_imgs_without_anomaly)

# X_pred2 = np.squeeze(x_pred2)
# Decoded_imgs_with_anomaly=np.squeeze(decoded_imgs_with_anomaly)

X_pred2 = np.squeeze(x_pred2)
Decoded_imgs_with_blob_anomaly=np.squeeze(decoded_imgs_with_blob_anomaly)
X_pred3 = np.squeeze(x_pred3)
Decoded_imgs_with_scratch_anomaly=np.squeeze(decoded_imgs_with_scratch_anomaly)

In [ ]:
print(X_pred1.shape)
print(Decoded_imgs_without_anomaly.shape)

In [ ]:
ssim_with_blob_anomaly_arr_window = np.asarray([get_error_ssim(X_pred2[i],Decoded_imgs_with_blob_anomaly[i]) for i in range(0, X_pred2.shape[0])])
ssim_with_scratch_anomaly_arr_window = np.asarray([get_error_ssim(X_pred3[i],Decoded_imgs_with_scratch_anomaly[i]) for i in range(0, X_pred3.shape[0])])
ssim_without_anomaly_arr_window = np.asarray([get_error_ssim(X_pred1[i],Decoded_imgs_without_anomaly [i]) for i in range(0, X_pred1.shape[0])])


In [ ]:
plt.figure()
plt.title("SSIM")
# plt.scatter(x=np.arange(0,len(ssim_with_anomaly_arr_window)),y=ssim_with_anomaly_arr_window, label='ssim with anomaly')
plt.scatter(x=np.arange(0,len(ssim_with_blob_anomaly_arr_window)),y=ssim_with_blob_anomaly_arr_window[:,0], label='ssim with blob anomaly')
plt.scatter(x=np.arange(0,len(ssim_with_scratch_anomaly_arr_window)),y=ssim_with_scratch_anomaly_arr_window[:,0], label='ssim with scratch anomaly')
plt.scatter(x=np.arange(0,len(ssim_without_anomaly_arr_window)),y=ssim_without_anomaly_arr_window[:,0], label='ssim without anomaly')
plt.legend()
plt.tight_layout()
plt.show()